In [1]:
import os
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import gc
import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
# df_train = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
# df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [4]:
try_test = '20190715_dist_ang_ori_bond_cos_test_{}.pickle'
trained_models = '20190715_2_champs_models_lgb_{}.pickle'
submission_name = '20190715_2submission.csv'
submission_temp = 'submission_temp_{}.pickle'

In [5]:
bond_types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN']

In [6]:
for i, b in enumerate(bond_types):
    print('-'*10, b, '-'*10)
    start = time.time()
    df_bond = pd.read_pickle(OUTPUT +try_test.format(b))
    y_pred_b =[]

    with open(OUTPUT + trained_models.format(b), 'rb') as f:
        models_b = pickle.load(f)

    X_test = df_bond[df_bond.columns[5:]].values  
    num_seed = len(models_b)
    for s in range(num_seed):
        gbm = models_b[s]
        y_pred_single = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        y_pred_b.append(y_pred_single)
        
    y_pred = np.array(y_pred_b).mean(axis=0)
    
    df_submit_b = pd.DataFrame([df_bond['id'], y_pred], index=['id', 'scalar_coupling_constant']).T
    df_submit_b.to_pickle(OUTPUT +submission_temp.format(b))
    
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    gc.collect()

---------- 1JHC ----------
elapsed_time:1197.4483001232147[sec]
---------- 2JHH ----------
elapsed_time:421.19561672210693[sec]
---------- 1JHN ----------
elapsed_time:14.156259536743164[sec]
---------- 2JHN ----------
elapsed_time:46.87821340560913[sec]
---------- 2JHC ----------
elapsed_time:1521.6596581935883[sec]
---------- 3JHH ----------
elapsed_time:893.0700449943542[sec]
---------- 3JHC ----------
elapsed_time:2036.7518043518066[sec]
---------- 3JHN ----------
elapsed_time:76.8850302696228[sec]


In [7]:
df_submit = pd.DataFrame()
for b in bond_types:
    df_submit_b = pd.read_pickle(OUTPUT +submission_temp.format(b))
    df_submit = pd.concat([df_submit, df_submit_b], axis=0)

In [8]:
df_submit['id'] = df_submit['id'].astype('int32')

In [9]:
df_submit.head()

,id,scalar_coupling_constant
0,4658148,196.898820
1,4658150,196.898820
2,4658152,90.810420
3,4658156,90.809981
4,4658159,82.820017


In [10]:
df_submit.to_csv(OUTPUT + submission_name, index=False)

In [11]:
b = "3JHC"
with open(OUTPUT + trained_models.format(b), 'rb') as f:
    gbm = pickle.load(f)[0]

In [12]:
pd.Series(gbm.feature_importance(), index=df_bond.columns[5:])[:50]

dist_H_0_x           124327
dist_H_1_x           123276
dist_H_2_x           103469
dist_H_3_x            86173
dist_H_4_x            71479
dist_H_5_x            54613
dist_H_6_x            41237
dist_H_7_x            29855
dist_H_8_x            21974
dist_H_9_x            14470
angle_H_0_x          116445
angle_H_1_x          115280
angle_H_2_x           97147
angle_H_3_x           78986
angle_H_4_x           66172
angle_H_5_x           52428
angle_H_6_x           39131
angle_H_7_x           27611
angle_H_8_x           19108
angle_H_9_x           11027
orientation_H_0_x     56877
orientation_H_1_x    106010
orientation_H_2_x     88432
orientation_H_3_x     73921
orientation_H_4_x     63215
orientation_H_5_x     53584
orientation_H_6_x     43755
orientation_H_7_x     30518
orientation_H_8_x     21516
orientation_H_9_x     12421
dist_C_0_x           143329
dist_C_1_x           151906
dist_C_2_x           163089
dist_C_3_x           148346
dist_C_4_x           125213
dist_C_5_x          

In [13]:
pd.Series(gbm.feature_importance(), index=df_bond.columns[5:])[50:100]

orientation_C_0_x      1746
orientation_C_1_x    101596
orientation_C_2_x    130521
orientation_C_3_x    129722
orientation_C_4_x    111393
orientation_C_5_x     82977
orientation_C_6_x     47728
orientation_C_7_x     16788
orientation_C_8_x      2930
orientation_C_9_x         0
bond_cos2_C_0_x      120862
bond_cos2_C_1_x      126764
bond_cos2_C_2_x      100579
bond_cos2_C_3_x       74725
bond_cos2_C_4_x       62702
bond_cos2_C_5_x       50219
bond_cos2_C_6_x       30166
bond_cos2_C_7_x       11146
bond_cos2_C_8_x        3084
bond_cos2_C_9_x           0
bond_cos3_C_0_x      134429
bond_cos3_C_1_x      132895
bond_cos3_C_2_x      105052
bond_cos3_C_3_x       84920
bond_cos3_C_4_x       72288
bond_cos3_C_5_x       57814
bond_cos3_C_6_x       36340
bond_cos3_C_7_x       15946
bond_cos3_C_8_x        2485
bond_cos3_C_9_x           0
dist_N_0_x           106428
dist_N_1_x            34943
dist_N_2_x            10545
angle_N_0_x           67557
angle_N_1_x           24749
angle_N_2_x         

In [14]:
pd.Series(gbm.feature_importance(), index=df_bond.columns[5:])[100:150]

angle_O_2_x           14002
orientation_O_0_x     91596
orientation_O_1_x     56144
orientation_O_2_x     10842
dist_F_0_x             2364
dist_F_1_x             1255
angle_F_0_x            1359
angle_F_1_x             683
orientation_F_0_x       770
orientation_F_1_x       344
dist_H_0_y           118208
dist_H_1_y           108936
dist_H_2_y            95338
dist_H_3_y            83933
dist_H_4_y            76695
dist_H_5_y            63604
dist_H_6_y            53352
dist_H_7_y            43005
dist_H_8_y            31053
dist_H_9_y            23488
angle_H_0_y           30363
angle_H_1_y           90856
angle_H_2_y           81577
angle_H_3_y           71268
angle_H_4_y           64114
angle_H_5_y           55426
angle_H_6_y           45089
angle_H_7_y           36931
angle_H_8_y           26777
angle_H_9_y           18834
orientation_H_0_y     39147
orientation_H_1_y     68223
orientation_H_2_y     80730
orientation_H_3_y     62785
orientation_H_4_y     54658
orientation_H_5_y   

In [15]:
pd.Series(gbm.feature_importance(), index=df_bond.columns[5:])[150:200]

angle_C_0_y          101221
angle_C_1_y          101437
angle_C_2_y          107286
angle_C_3_y           92453
angle_C_4_y           70308
angle_C_5_y           40750
angle_C_6_y           14239
angle_C_7_y            2232
angle_C_8_y               0
angle_C_9_y               0
orientation_C_0_y     67977
orientation_C_1_y     88575
orientation_C_2_y     90885
orientation_C_3_y     73952
orientation_C_4_y     53649
orientation_C_5_y     29460
orientation_C_6_y      9952
orientation_C_7_y      2001
orientation_C_8_y         0
orientation_C_9_y         0
bond_cos2_C_0_y      113075
bond_cos2_C_1_y       99891
bond_cos2_C_2_y       83176
bond_cos2_C_3_y       67048
bond_cos2_C_4_y       48756
bond_cos2_C_5_y       27357
bond_cos2_C_6_y       10319
bond_cos2_C_7_y        2898
bond_cos2_C_8_y           0
bond_cos2_C_9_y           0
bond_cos3_C_0_y      115890
bond_cos3_C_1_y       99941
bond_cos3_C_2_y       87628
bond_cos3_C_3_y       78603
bond_cos3_C_4_y       61452
bond_cos3_C_5_y     

In [16]:
pd.Series(gbm.feature_importance(), index=df_bond.columns[5:])[200:]

bond_cos2_N_1_y       14877
bond_cos2_N_2_y        5039
bond_cos3_N_0_y       28586
bond_cos3_N_1_y       10288
bond_cos3_N_2_y        3410
dist_O_0_y           159778
dist_O_1_y            80279
dist_O_2_y            17312
angle_O_0_y           77092
angle_O_1_y           43294
angle_O_2_y            8286
orientation_O_0_y     60513
orientation_O_1_y     39106
orientation_O_2_y      8953
dist_F_0_y             3521
dist_F_1_y             1511
angle_F_0_y             249
angle_F_1_y             218
orientation_F_0_y       581
orientation_F_1_y       569
cos_3j               321159
cos_3j^2             238135
dist_center          236486
dtype: int64